In [2]:
!pip install transformers accelerate tqdm

In [16]:
from tqdm import tqdm
import accelerate
from transformers import pipeline

import numpy as np
from scipy.special import softmax
import pandas as pd

In [21]:
# Model Names
MODEL_DE = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
MODEL_UK = f"oliverguhr/german-sentiment-bert"

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)




In [19]:
def get_score(text):
    result = sentiment_task(text)
    result = result[0]['score'] if result[0]['label'] =='positive' else -result[0]['score']
    return result

In [24]:
sentiment_task = pipeline("sentiment-analysis", model=MODEL_UK, tokenizer=MODEL_UK, device=0)
data = pd.read_csv("results/uk_sample.csv")
data['clean_text'] = data['text'].apply(lambda x: preprocess(x))
tqdm.pandas()
data['emotional_tone'] = data['clean_text'].progress_apply(lambda x: get_score(x))
data.to_csv("results/uk_sample_sent.csv")

  0%|          | 10/260000 [00:00<43:23, 99.87it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 260000/260000 [48:26<00:00, 89.47it/s] 


In [26]:
sentiment_task = pipeline("sentiment-analysis", model=MODEL_DE, tokenizer=MODEL_DE, device=0)
data = pd.read_csv("results/de_sample.csv")
data['clean_text'] = data['text'].apply(lambda x: preprocess(x))
tqdm.pandas()
data['emotional_tone'] = data['clean_text'].progress_apply(lambda x: get_score(x))
data.to_csv("/kaggle/working/de_sampe_sent.csv")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 10/260000 [00:00<45:03, 96.15it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2